In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%watermark -v -p tensorflow,tensorboard
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.5
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
CPython 3.6.9
IPython 7.16.1

tensorflow 2.6.0
tensorboard 2.6.0


In [2]:
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'tensorboard-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_cell_magic
def template_writefile(line, cell):
    with open(line, 'w') as fw:
        fw.write(cell.format(**globals()))

@register_line_magic
def netron(line):
    args = line.split()
    file, port, height = args[0], int(args[1]), 600
    if len(args) == 3:
        height = int(args[2])
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #     pid = int(res[0].split(' ')[1])
    #     !kill -9 $pid
    import netron
    netron.start(file, address=('0.0.0.0', port), browse=False)
    display_html(port, height)


In [24]:
#@title
import base64
import io
import os
import time

from scipy.signal import medfilt
import cv2


from IPython.display import display
from IPython.display import HTML
from IPython.display import Javascript

import matplotlib
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image


# import tensorflow.compat.v2 as tf
import tensorflow as tf

# ! pip install youtube_dl
# import youtube_dl

# from google.colab import drive
# from google.colab import output
# from google.colab.output import eval_js

# Model definition
layers = tf.keras.layers
regularizers = tf.keras.regularizers


def get_sims(embs, temperature):
  """Calculates self-similarity between batch of sequence of embeddings."""
  batch_size = tf.shape(embs)[0]
  seq_len = tf.shape(embs)[1]
  embs = tf.reshape(embs, [batch_size, seq_len, -1])

  def _get_sims(embs):
    """Calculates self-similarity between sequence of embeddings."""
    dist = pairwise_l2_distance(embs, embs)
    sims = -1.0 * dist
    return sims

  sims = tf.map_fn(_get_sims, embs)
  sims /= temperature
  sims = tf.nn.softmax(sims, axis=-1)
  sims = tf.expand_dims(sims, -1)
  return sims


def flatten_sequential_feats(x, batch_size, seq_len):
  """Flattens sequential features with known batch size and seq_len."""
  x = tf.reshape(x, [batch_size, seq_len, -1])
  return x


# Transformer from https://www.tensorflow.org/tutorials/text/transformer .
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.

  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    outputs: shape == (..., seq_len_q, depth_v)
    attention_weights: shape == (..., seq_len_q, seq_len_k)
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk.
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  # (..., seq_len_q, seq_len_k)
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

  outputs = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return outputs, attention_weights


def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])


class MultiHeadAttention(tf.keras.layers.Layer):
  """Multi-headed attention layer."""

  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth)."""
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(
        q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(
        k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(
        v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(
        scaled_attention,
        perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(
        scaled_attention,
        (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    outputs = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return outputs, attention_weights


class TransformerLayer(tf.keras.layers.Layer):
  """Implements a single transformer layer (https://arxiv.org/abs/1706.03762).
  """

  def __init__(self, d_model, num_heads, dff,
               dropout_rate=0.1,
               reorder_ln=False):
    super(TransformerLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.reorder_ln = reorder_ln

  def call(self, x):
    inp_x = x

    if self.reorder_ln:
      x = self.layernorm1(x)

    # (batch_size, input_seq_len, d_model)
    attn_output, _ = self.mha(x, x, x, mask=None)
    attn_output = self.dropout1(attn_output)

    if self.reorder_ln:
      out1 = inp_x + attn_output
      x = out1
    else:
      # (batch_size, input_seq_len, d_model)
      out1 = self.layernorm1(x + attn_output)
      x = out1

    if self.reorder_ln:
      x = self.layernorm2(x)

    # (batch_size, input_seq_len, d_model)
    ffn_output = self.ffn(x)
    ffn_output = self.dropout2(ffn_output)

    if self.reorder_ln:
      out2 = out1 + ffn_output
    else:
      # (batch_size, input_seq_len, d_model)
      out2 = self.layernorm2(out1 + ffn_output)

    return out2


def pairwise_l2_distance(a, b):
  """Computes pairwise distances between all rows of a and all rows of b."""
  norm_a = tf.reduce_sum(tf.square(a), 1)
  norm_a = tf.reshape(norm_a, [-1, 1])
  norm_b = tf.reduce_sum(tf.square(b), 1)
  norm_b = tf.reshape(norm_b, [1, -1])
  dist = tf.maximum(norm_a - 2.0 * tf.matmul(a, b, False, True) + norm_b, 0.0)
  return dist


def get_repnet_model(logdir):
  """Returns a trained RepNet model.

  Args:
    logdir (string): Path to directory where checkpoint will be downloaded.

  Returns:
    model (Keras model): Trained RepNet model.
  """
  # Check if we are in eager mode.
  assert tf.executing_eagerly()

  # Models will be called in eval mode.
  tf.keras.backend.set_learning_phase(0)

  # Define RepNet model.
  model = ResnetPeriodEstimator()
  # tf.function for speed.
  model.call = tf.function(model.call)

  # Define checkpoint and checkpoint manager.
  ckpt = tf.train.Checkpoint(model=model)
  ckpt_manager = tf.train.CheckpointManager(
      ckpt, directory=logdir, max_to_keep=10)
  latest_ckpt = ckpt_manager.latest_checkpoint
  print('Loading from: ', latest_ckpt)
  if not latest_ckpt:
    raise ValueError('Path does not have a checkpoint to load.')
  # Restore weights.
  ckpt.restore(latest_ckpt).expect_partial()

  # Pass dummy frames to build graph.
  model(tf.random.uniform((1, 64, 112, 112, 3)))
  return model


def unnorm(query_frame):
  min_v = query_frame.min()
  max_v = query_frame.max()
  query_frame = (query_frame - min_v) / max(1e-7, (max_v - min_v))
  return query_frame

class ResnetPeriodEstimator(tf.keras.models.Model):
  """RepNet model."""

  def __init__(
      self,
      num_frames=64,
      image_size=112,
      base_model_layer_name='conv4_block3_out',
      temperature=13.544,
      dropout_rate=0.25,
      l2_reg_weight=1e-6,
      temporal_conv_channels=512,
      temporal_conv_kernel_size=3,
      temporal_conv_dilation_rate=3,
      conv_channels=32,
      conv_kernel_size=3,
      transformer_layers_config=((512, 4, 512),),
      transformer_dropout_rate=0.0,
      transformer_reorder_ln=True,
      period_fc_channels=(512, 512),
      within_period_fc_channels=(512, 512)):
    super(ResnetPeriodEstimator, self).__init__()

    # Model params.
    self.num_frames = num_frames
    self.image_size = image_size

    self.base_model_layer_name = base_model_layer_name

    self.temperature = temperature

    self.dropout_rate = dropout_rate
    self.l2_reg_weight = l2_reg_weight

    self.temporal_conv_channels = temporal_conv_channels
    self.temporal_conv_kernel_size = temporal_conv_kernel_size
    self.temporal_conv_dilation_rate = temporal_conv_dilation_rate

    self.conv_channels = conv_channels
    self.conv_kernel_size = conv_kernel_size
    # Transformer config in form of (channels, heads, bottleneck channels).
    self.transformer_layers_config = transformer_layers_config
    self.transformer_dropout_rate = transformer_dropout_rate
    self.transformer_reorder_ln = transformer_reorder_ln

    self.period_fc_channels = period_fc_channels
    self.within_period_fc_channels = within_period_fc_channels

    # Base ResNet50 Model.
    base_model = tf.keras.applications.ResNet50V2(
        include_top=False, weights=None, pooling='max')
    self.base_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=base_model.get_layer(self.base_model_layer_name).output)

    # 3D Conv on k Frames
    self.temporal_conv_layers = [
        layers.Conv3D(self.temporal_conv_channels,
                      self.temporal_conv_kernel_size,
                      padding='same',
                      dilation_rate=(self.temporal_conv_dilation_rate, 1, 1),
                      kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                      kernel_initializer='he_normal', name='RE_temporal_conv_layers')]
    self.temporal_bn_layers = [layers.BatchNormalization(name='RE_temporal_bn_layers')
                               for _ in self.temporal_conv_layers]

    # Counting Module (Self-sim > Conv > Transformer > Classifier)
    self.conv_3x3_layer = layers.Conv2D(self.conv_channels,
                                        self.conv_kernel_size,
                                        padding='same',
                                        activation=tf.nn.relu, name='RE_conv_3x3_layer')

    channels = self.transformer_layers_config[0][0]
    self.input_projection = layers.Dense(
        channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
        activation=None, name='RE_input_projection')

    self.input_projection2 = layers.Dense(
        channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
        activation=None, name='RE_input_projection2')

    length = self.num_frames
    self.pos_encoding = tf.compat.v1.get_variable(
        name='resnet_period_estimator/pos_encoding',
        shape=[1, length, 1],
        initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))
    self.pos_encoding2 = tf.compat.v1.get_variable(
        name='resnet_period_estimator/pos_encoding2',
        shape=[1, length, 1],
        initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))

    self.transformer_layers = []
    for d_model, num_heads, dff in self.transformer_layers_config:
      self.transformer_layers.append(
          TransformerLayer(d_model, num_heads, dff,
                           self.transformer_dropout_rate,
                           self.transformer_reorder_ln))

    self.transformer_layers2 = []
    for d_model, num_heads, dff in self.transformer_layers_config:
      self.transformer_layers2.append(
          TransformerLayer(d_model, num_heads, dff,
                           self.transformer_dropout_rate,
                           self.transformer_reorder_ln))

    # Period Prediction Module.
    self.dropout_layer = layers.Dropout(self.dropout_rate, name='RE_dropout_layer')
    num_preds = self.num_frames//2
    self.fc_layers = []
    for i, channels in enumerate(self.period_fc_channels):
      self.fc_layers.append(layers.Dense(
          channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
          activation=tf.nn.relu, name=f'RE_fc_layers_{i}'))
    self.fc_layers.append(layers.Dense(
        num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight), name='RE_fc_layers_x'))

    # Within Period Module
    num_preds = 1
    self.within_period_fc_layers = []
    for i, channels in enumerate(self.within_period_fc_channels):
      self.within_period_fc_layers.append(layers.Dense(
          channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
          activation=tf.nn.relu, name=f'RE_within_period_fc_layers_{i}'))
    self.within_period_fc_layers.append(layers.Dense(
        num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight), name='RE_within_period_fc_layers_x'))

  def call(self, x):
    # Ensures we are always using the right batch_size during train/eval.
    batch_size = tf.shape(x)[0]
    # Conv Feature Extractor.
    print('input x:', x.shape)
    x = tf.reshape(x, [-1, self.image_size, self.image_size, 3])
    print('before base_model', x.shape)
    x = self.base_model(x)
    print('after base_model', x.shape)
    h = tf.shape(x)[1]
    w = tf.shape(x)[2]
    c = tf.shape(x)[3]
    x = tf.reshape(x, [batch_size, -1, h, w, c])
    print('reshape:', x.shape)

    # 3D Conv to give temporal context to per-frame embeddings. 
    for bn_layer, conv_layer in zip(self.temporal_bn_layers,
                                    self.temporal_conv_layers):
      x = conv_layer(x)
      x = bn_layer(x)
      x = tf.nn.relu(x)
    
    print('temporal shape:', x.shape)

    x = tf.reduce_max(x, [2, 3])
    
    print('reduce max shape:', x.shape)

    # Reshape and prepare embs for output.
    final_embs = x

    # Get self-similarity matrix.
    x = get_sims(x, self.temperature)
    
    print('sims x:', x.shape)

    # 3x3 conv layer on self-similarity matrix.
    x = self.conv_3x3_layer(x)
    print('before reshape: ', x.shape)
    x = tf.reshape(x, [batch_size, self.num_frames, -1])
    print('after reshape', x.shape)
    within_period_x = x

    # Period prediction.
    x = self.input_projection(x)
    print('input_projection:', x.shape)
    # x = tf.add(x, self.pos_encoding)
    x += self.pos_encoding
    for transformer_layer in self.transformer_layers:
      x = transformer_layer(x)
    x = flatten_sequential_feats(x, batch_size, self.num_frames)
    for fc_layer in self.fc_layers:
      x = self.dropout_layer(x)
      x = fc_layer(x)

    # Within period prediction.
    within_period_x = self.input_projection2(within_period_x)
    within_period_x += self.pos_encoding2
    for transformer_layer in self.transformer_layers2:
      within_period_x = transformer_layer(within_period_x)
    within_period_x = flatten_sequential_feats(within_period_x,
                                               batch_size,
                                               self.num_frames)
    for fc_layer in self.within_period_fc_layers:
      within_period_x = self.dropout_layer(within_period_x)
      within_period_x = fc_layer(within_period_x)

    return x, within_period_x, final_embs

  @tf.function
  def preprocess(self, imgs):
    imgs = tf.cast(imgs, tf.float32)
    imgs -= 127.5
    imgs /= 127.5
    imgs = tf.image.resize(imgs, (self.image_size, self.image_size))
    return imgs

In [33]:
a = tf.random.uniform((1, 4, 10))
b = tf.random.uniform((1, 4, 1))
a, b

(<tf.Tensor: shape=(1, 4, 10), dtype=float32, numpy=
 array([[[0.8214896 , 0.97153926, 0.31927598, 0.90383804, 0.56659114,
          0.3860495 , 0.03729355, 0.51917446, 0.5181935 , 0.9286331 ],
         [0.17982829, 0.1925621 , 0.5589218 , 0.6346531 , 0.32540655,
          0.186571  , 0.161443  , 0.4177512 , 0.9843551 , 0.502751  ],
         [0.30991375, 0.36333644, 0.0285337 , 0.5291929 , 0.6247728 ,
          0.24720311, 0.17142904, 0.3545364 , 0.85511076, 0.31896532],
         [0.7604172 , 0.3044914 , 0.703091  , 0.45360887, 0.7035121 ,
          0.08286226, 0.99000204, 0.37165272, 0.19908941, 0.10229671]]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 4, 1), dtype=float32, numpy=
 array([[[0.28793263],
         [0.02346492],
         [0.05378044],
         [0.5619633 ]]], dtype=float32)>)

In [34]:
tf.tile(b, multiples=[1, 1, 10])

<tf.Tensor: shape=(1, 4, 10), dtype=float32, numpy=
array([[[0.28793263, 0.28793263, 0.28793263, 0.28793263, 0.28793263,
         0.28793263, 0.28793263, 0.28793263, 0.28793263, 0.28793263],
        [0.02346492, 0.02346492, 0.02346492, 0.02346492, 0.02346492,
         0.02346492, 0.02346492, 0.02346492, 0.02346492, 0.02346492],
        [0.05378044, 0.05378044, 0.05378044, 0.05378044, 0.05378044,
         0.05378044, 0.05378044, 0.05378044, 0.05378044, 0.05378044],
        [0.5619633 , 0.5619633 , 0.5619633 , 0.5619633 , 0.5619633 ,
         0.5619633 , 0.5619633 , 0.5619633 , 0.5619633 , 0.5619633 ]]],
      dtype=float32)>

In [6]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [39]:
model1 = ResnetPeriodEstimator()
print(model1.pos_encoding.shape)
print(model1.pos_encoding[0][0][:10])
ckpt = tf.train.Checkpoint(model=model1)
ckpt.restore('/data/pretrained/cv/repnet/ckpt-88').expect_partial()
print('-'*20)
model1.pos_encoding = tf.tile(model1.pos_encoding, multiples=[1, 1, 512])
model1.pos_encoding2 = tf.tile(model1.pos_encoding2, multiples=[1, 1, 512])
print(model1.pos_encoding.shape)
print(model1.pos_encoding[0][0][:10]) # 0.02054866
test_inputs = tf.random.uniform((1, 64, 112, 112, 3))
model1(test_inputs)
predictions = model1.predict(test_inputs)
model1.summary()

(1, 64, 1)
tf.Tensor([0.00110708], shape=(1,), dtype=float32)
--------------------
(1, 64, 512)
tf.Tensor(
[0.02054866 0.02054866 0.02054866 0.02054866 0.02054866 0.02054866
 0.02054866 0.02054866 0.02054866 0.02054866], shape=(10,), dtype=float32)
input x: (1, 64, 112, 112, 3)
before base_model (64, 112, 112, 3)
after base_model (64, 7, 7, 1024)
reshape: (1, 64, 7, 7, 1024)
temporal shape: (1, 64, 7, 7, 512)
reduce max shape: (1, 64, 512)
sims x: (1, 64, 64, 1)
before reshape:  (1, 64, 64, 32)
after reshape (1, 64, 2048)
input_projection: (1, 64, 512)
input x: (None, 64, 112, 112, 3)
before base_model (None, 112, 112, 3)
after base_model (None, 7, 7, 1024)
reshape: (None, None, None, None, None)
temporal shape: (None, None, None, None, 512)
reduce max shape: (None, None, 512)
sims x: (None, None, None, 1)
before reshape:  (None, None, None, 32)
after reshape (None, 64, None)
input_projection: (None, 64, 512)
Model: "resnet_period_estimator_7"
__________________________________________

In [30]:
model1.pos_encoding.shape

TensorShape([1, 64, 1])

In [40]:
full_model = tf.function(lambda x: model1(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(test_inputs.shape, test_inputs.dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
# frozen_func.graph.as_graph_def()

input x: (1, 64, 112, 112, 3)
before base_model (64, 112, 112, 3)
after base_model (64, 7, 7, 1024)
reshape: (1, 64, 7, 7, 1024)
temporal shape: (1, 64, 7, 7, 512)
reduce max shape: (1, 64, 512)
sims x: (1, 64, 64, 1)
before reshape:  (1, 64, 64, 32)
after reshape (1, 64, 2048)
input_projection: (1, 64, 512)


In [18]:
graph = frozen_func.graph.as_graph_def()

In [19]:
operations = frozen_func.graph.get_operations()

In [20]:
print("-" * 50)
print("Frozen model layers: ")
for op in operations:
    print(op.name)

--------------------------------------------------
Frozen model layers: 
x
resnet_period_estimator_1/Shape_4
resnet_period_estimator_1/strided_slice_4/stack
resnet_period_estimator_1/strided_slice_4/stack_1
resnet_period_estimator_1/strided_slice_4/stack_2
resnet_period_estimator_1/strided_slice_4
resnet_period_estimator_1/Shape_5
resnet_period_estimator_1/strided_slice_5/stack
resnet_period_estimator_1/strided_slice_5/stack_1
resnet_period_estimator_1/strided_slice_5/stack_2
resnet_period_estimator_1/strided_slice_5
resnet_period_estimator_1/Reshape_2/shape/2
resnet_period_estimator_1/Reshape_2/shape
unused_control_flow_input
resnet_period_estimator_1/Reshape_2
resnet_period_estimator_1/map/TensorArrayUnstack/TensorListFromTensor/element_shape
resnet_period_estimator_1/map/TensorArrayUnstack/TensorListFromTensor
resnet_period_estimator_1/map/while/enter/_6
unused_control_flow_input_1
Func/resnet_period_estimator_1/map/while/body/_1/output/_47
resnet_period_estimator_1/map/while/next_i

resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_24/Tensordot
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_24/BiasAdd/ReadVariableOp/resource
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_24/BiasAdd/ReadVariableOp
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_24/BiasAdd
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_24/Relu
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_25/Tensordot/Reshape/shape
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_25/Tensordot/Reshape
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_25/Tensordot/ReadVariableOp/resource
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_25/Tensordot/ReadVariableOp
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_25/Tensordot/MatMul
resnet_period_estimator_1/transformer_layer_2/sequential_2/dense_25/Tensordot/shape
resnet_period_estimator_1/transformer_la

##### tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir="/data/nb_data/tmp/frozen_models",
    name="simple_frozen_graph3.pb",
    as_text=False)

In [42]:
!python3 -m tf2onnx.convert --input /data/nb_data/tmp/frozen_models/simple_frozen_graph3.pb \
    --inputs x:0 --outputs Identity:0,Identity_1:0,Identity_2:0 \
    --opset 13 --output /data/nb_data/tmp/repnet3.ox

2021-08-27 23:00:51.803125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-08-27 23:00:51.803218: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-08-27 23:00:53.630129: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-27 23:00:53.630194: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: k12nb
2021-

In [43]:
%netron /data/nb_data/tmp/repnet3.onnx 8222

Stopping http://0.0.0.0:8222
Serving '/data/nb_data/tmp/repnet3.onnx' at http://0.0.0.0:8222


----------------------------------------
Exception happened during processing of request from ('74.120.14.43', 51628)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 654, in process_request_thread
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 386, in handle_one_request
    self.raw_requestline = self.rfile.readline(65537)
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer
----------------------------------------
----------------------------------------
Exception happened during processing o

In [ ]:
!python3 -m tf2onnx.convert --tag serve --signature_def serving_default --opset 13 \
    --saved-model /data/ff --output /data/ff/test.onnx --verbose --debug